In [2]:
import pandas as pd
import numpy as np
import clean
import model

# Exploracion de datos

In [ ]:
df = pd.read_csv("data/train_df.csv")

In [ ]:
print(df["target"].value_counts(normalize=True))

target
0    0.504667
1    0.495333
Name: proportion, dtype: float64


In [14]:
print(df.describe())


       ratio_colesterol  actividad_fisica  presion_arterial  nivel_glucosa  \
count       4500.000000       4500.000000       4500.000000    4500.000000   
mean         167.510934         16.122076        129.798259     289.652186   
std           20.017701          6.309545         17.698567      76.170775   
min          114.654997        -13.243917         72.443614     117.432144   
25%          149.893673         12.381921        117.488555     236.281633   
50%          167.992048         16.744957        129.605852     281.245188   
75%          184.992202         20.455692        141.884810     336.292962   
max          217.617202         34.035411        184.892159     720.852288   

       indice_masa_corporal  horas_sueno  historial_diabetes  \
count           4500.000000  4500.000000         4500.000000   
mean              25.092246     6.991862            0.200222   
std                5.045592     1.743349            0.400211   
min                6.474250     4.002980 

In [15]:
print(df.isnull().sum())


paciente_id             0
ratio_colesterol        0
actividad_fisica        0
presion_arterial        0
nivel_glucosa           0
indice_masa_corporal    0
horas_sueno             0
historial_diabetes      0
frecuencia_cardiaca     0
proteina_c_reactiva     0
dias_ultima_consulta    0
consumo_alcohol         0
edad                    0
genero                  0
nivel_estres            0
target                  0
dtype: int64


In [16]:
print(df.head())


  paciente_id  ratio_colesterol  actividad_fisica  presion_arterial  \
0   PAC_11952        178.752967          6.747645        114.971011   
1   PAC_10214        168.330154         23.521691         95.529973   
2   PAC_09998        185.735985          0.224380        163.923803   
3   PAC_00130        178.513403         13.487034        147.598815   
4   PAC_10168        153.748651         15.638448        126.253096   

   nivel_glucosa  indice_masa_corporal  horas_sueno  historial_diabetes  \
0     151.008232             15.852293     5.110575                   0   
1     297.313974             27.482569     8.090175                   0   
2     286.279129             24.705224     8.680475                   1   
3     351.784306             28.917964     8.405222                   1   
4     213.042582             17.779001     6.533278                   0   

   frecuencia_cardiaca  proteina_c_reactiva  dias_ultima_consulta  \
0            22.121619             6.991028          

In [17]:
print(df.corr(numeric_only=True)["target"].sort_values(ascending=False))

target                  1.000000
horas_sueno             0.019934
consumo_alcohol         0.018628
proteina_c_reactiva     0.009160
ratio_colesterol        0.005497
nivel_estres            0.001612
actividad_fisica        0.001469
frecuencia_cardiaca    -0.000867
dias_ultima_consulta   -0.002478
edad                   -0.004104
indice_masa_corporal   -0.016382
nivel_glucosa          -0.019268
historial_diabetes     -0.033649
presion_arterial       -0.038872
Name: target, dtype: float64


Las variables tienen una relacion muy baja con target. Esto indica que la relacion puede no ser lineal.

In [3]:
# 1. Cargar datos de entrenamiento para calcular media y std
train_df = pd.read_csv("data/train_df.csv")
X_train, y_train, media, std = clean.prep(train_df)

# 2. Entrenar modelo con todos los datos
weights = model.train(X_train, y_train, lr=0.1, epochs=1000)

# 3. Cargar datos de test
test_df = pd.read_csv("data/train_df.csv")
paciente_ids = test_df["paciente_id"].values
y_true = test_df["target"].values

# 4. Preprocesar test (sin target)
def prep_test(df, media, std):
    df = df.copy()
    df = df.drop(columns=["paciente_id"])
    df["genero"] = df["genero"].map({"F": 0, "M": 1})
    X = df.drop(columns=["target"]).values
    X_normalizado = (X - media) / std
    X_final = clean.add_bias(X_normalizado)
    return X_final

X_test = prep_test(test_df, media, std)

In [4]:
# 5. Predecir
y_pred = model.predict(X_test, weights)

# 6. Crear y guardar DataFrame
predicciones = pd.DataFrame({
    "paciente_id": paciente_ids,
    "target": y_pred.astype(int)
})
predicciones.to_csv("predicciones.csv", index=False)
print("Archivo guardado como predicciones.csv")

Archivo guardado como predicciones.csv


In [5]:
precision, recall, f1 = model.precision_recall_f1(y_train, y_pred)
print(f"Precisión: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Precisión: 0.5261
Recall: 0.4832
F1-Score: 0.5037


In [28]:
import numpy as np
from model import train, predict
from clean import add_bias, k_fold_split
from sklearn.metrics import f1_score

# Transformaciones no lineales seguras
def log(x): return np.log1p(np.maximum(x, 0))
def sqrt(x): return np.sqrt(np.maximum(x, 0))
def square(x): return np.power(x, 2)
def identity(x): return x

transformaciones = {
    "log": log,
    "sqrt": sqrt,
    "square": square,
    "identity": identity
}

def transformar(X, transformaciones):
    X_new = np.zeros_like(X)
    for i, t in enumerate(transformaciones):
        if t == "original":
            X_new[:, i] = X[:, i]
        elif t == "cuadrado":
            X_new[:, i] = X[:, i] ** 2
        elif t == "raiz":
            X_new[:, i] = np.sqrt(np.maximum(X[:, i], 0))
        elif t == "log":
            X_new[:, i] = np.log1p(np.maximum(X[:, i], 0))
    return X_new

def evaluar_mejores_transformaciones(X, y):
    opciones = ["original", "cuadrado", "raiz", "log"]
    mejores = ["original"] * X.shape[1]

    for i in range(X.shape[1]):
        mejor_score = -1
        mejor_transf = "original"

        for t in opciones:
            temp = mejores.copy()
            temp[i] = t
            X_trans = transformar(X, temp)
            X_norm = (X_trans - X_trans.mean(axis=0)) / X_trans.std(axis=0)
            X_final = clean.add_bias(X_norm)
            weights = model.train(X_final, y, lr=0.1, epochs=500)
            y_pred = model.predict(X_final, weights)
            f1 = model.f1_score(y, y_pred)

            if f1 > mejor_score:
                mejor_score = f1
                mejor_transf = t

        mejores[i] = mejor_transf

    # aplicar la mejor combinación final
    X_mejor = transformar(X, mejores)
    return X_mejor, mejores


In [29]:
# 1. Cargar datos
df = pd.read_csv("data/train_df.csv")

# 2. Preprocesamiento inicial (como en clean.prep)
df = df.drop(columns=["paciente_id"])
df["genero"] = df["genero"].map({"F": 0, "M": 1})
y = df["target"].values
X_raw = df.drop(columns=["target"]).values  # sin normalizar aún


In [30]:
# 3. Buscar mejores transformaciones automáticamente
X_trans, mejores_transformaciones = evaluar_mejores_transformaciones(X_raw, y)

# 4. Normalizar y agregar bias
media = X_trans.mean(axis=0)
std = X_trans.std(axis=0)
X_normalizado = (X_trans - media) / std
X_final = clean.add_bias(X_normalizado)

In [31]:
# 5. Entrenar modelo
weights = model.train(X_final, y, lr=0.1, epochs=1000)

# 6. Predecir sobre mismo conjunto (simulación de test)
y_pred = model.predict(X_final, weights)


In [35]:
# 7. Evaluar
precision, recall, f1 = model.precision_recall_f1(y, y_pred)

print(f"Precisión: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# 8. Mostrar transformaciones usadas
print("\nMejores transformaciones por variable:")
for i, name in enumerate(df.drop(columns=["target"]).columns):
    print(f"{name}: {mejores_transformaciones[i]}")

Precisión: 0.5297
Recall: 0.4998
F1-Score: 0.5143

Mejores transformaciones por variable:
ratio_colesterol: log
actividad_fisica: cuadrado
presion_arterial: cuadrado
nivel_glucosa: original
indice_masa_corporal: original
horas_sueno: original
historial_diabetes: original
frecuencia_cardiaca: raiz
proteina_c_reactiva: original
dias_ultima_consulta: log
consumo_alcohol: cuadrado
edad: raiz
genero: original
nivel_estres: cuadrado
